## Import Libraries and Load Data

In [1]:
# libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np
pio.templates.default = "plotly_white"

# read csv file of data
data = pd.read_csv('rides.csv')
print(data.head())

   Drivers Active Per Hour  Riders Active Per Hour  Rides Completed
0                       72                     295            202.0
1                       50                      78             43.0
2                       40                     250            181.0
3                       78                     140            124.0
4                       74                     195            108.0


In [2]:
# check total number of null values in each column
print(data.isnull().sum())

Drivers Active Per Hour     0
Riders Active Per Hour      0
Rides Completed            54
dtype: int64


In [3]:
# drop null rows
data = data.dropna()

In [4]:
# analysis demands of riders
demand = data["Riders Active Per Hour"]
supply = data["Drivers Active Per Hour"]

figure = px.scatter(data, x = "Drivers Active Per Hour",
                    y = "Riders Active Per Hour", trendline="ols", 
                    title="Demand and Supply Analysis")
figure.update_layout(
    xaxis_title="Number of Drivers Active per Hour (Supply)",
    yaxis_title="Number of Riders Active per Hour (Demand)",
)
figure.show()

In [5]:
# Calculate elasticity
avg_demand = data['Riders Active Per Hour'].mean()
avg_supply = data['Drivers Active Per Hour'].mean()
pct_change_demand = (max(data['Riders Active Per Hour']) - min(data['Riders Active Per Hour'])) / avg_demand * 100
pct_change_supply = (max(data['Drivers Active Per Hour']) - min(data['Drivers Active Per Hour'])) / avg_supply * 100
elasticity = pct_change_demand / pct_change_supply

print("Elasticity of demand with respect to the number of active drivers per hour: {:.2f}".format(elasticity))

Elasticity of demand with respect to the number of active drivers per hour: 0.82


In [6]:
# Calculate the supply ratio for each level of driver activity
data['Supply Ratio'] = data['Rides Completed'] / data['Drivers Active Per Hour']
print(data.head())

   Drivers Active Per Hour  Riders Active Per Hour  Rides Completed  \
0                       72                     295            202.0   
1                       50                      78             43.0   
2                       40                     250            181.0   
3                       78                     140            124.0   
4                       74                     195            108.0   

   Supply Ratio  
0      2.805556  
1      0.860000  
2      4.525000  
3      1.589744  
4      1.459459  


In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['Drivers Active Per Hour'], 
                         y=data['Supply Ratio'], mode='markers'))
fig.update_layout(
    title='Supply Ratio vs. Driver Activity',
    xaxis_title='Driver Activity (Drivers Active Per Hour)',
    yaxis_title='Supply Ratio (Rides Completed per Driver Active per Hour)'
)
fig.show()

In [8]:
# independent and dependent features
X = data.drop('Supply Ratio', axis=1)
y=data['Supply Ratio']

In [9]:
# split data for training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [10]:
# scale down independent features
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

## SVM Model

In [11]:
# apply svr model
from sklearn.svm import SVR

regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

SVR()

In [12]:
# predict on test data
y_pred = regressor.predict(X_test)

One for the actual data points represented as markers, and another for the regression line represented as a line connecting the predicted values

In [13]:
# two traces
fig = go.Figure()

fig.add_trace(go.Scatter(x=X_test.flatten(), y=y_test, mode='markers', name='Data points'))
fig.add_trace(go.Scatter(x=X_train.flatten(), y=y_pred.flatten(), mode='lines', name='Regression line'))

fig.update_layout(
    title='Truth or Bluff (SVR)',
    xaxis_title='Position level',
    yaxis_title='Salary'
)

fig.show()